# Notebook 2: Model Training & Hyperparameter Optimization
## Method A - Sentiment Analysis with Domain Adaptation

### Objectives:
1. Train TF-IDF + FFNN classifier
2. Train E5/MiniLM embedding classifier
3. Fine-tune BERT for sentiment analysis
4. Hyperparameter optimization with Optuna
5. Save best models for evaluation

### Models to Train:
- **Model 1**: TF-IDF features + Feed-Forward Neural Network
- **Model 2**: E5/MiniLM embeddings + Classifier head
- **Model 3**: BERT fine-tuning (bert-base-uncased)

### Training Strategy:
- Train on: IMDB + Yelp combined (multi-domain)
- Validate on: IMDB + Yelp validation split
- Test on: Amazon (cross-domain) - in Notebook 3

---

In [1]:
# Install required packages
# Run this cell only once
%pip install torch transformers sentence-transformers scikit-learn optuna tqdm numpy pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
## Setup & Imports - Part 1: Core Libraries

import os
import json
import pickle
import random
from pathlib import Path
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

print("✓ Core libraries loaded")

✓ Core libraries loaded


In [3]:
## Setup & Imports - Part 2: PyTorch & Deep Learning
print("Loading PyTorch... (this may take 30-60 seconds)")

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

print(f"✓ PyTorch loaded (version: {torch.__version__})")
print(f"  CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")

Loading PyTorch... (this may take 30-60 seconds)
✓ PyTorch loaded (version: 2.4.1+cu121)
  CUDA available: True
  GPU: NVIDIA A100 80GB PCIe
✓ PyTorch loaded (version: 2.4.1+cu121)
  CUDA available: True
  GPU: NVIDIA A100 80GB PCIe


In [4]:
## Additional imports for K-Fold Cross-Validation
from sklearn.model_selection import StratifiedKFold
import copy

print("✓ K-Fold CV imports loaded")

✓ K-Fold CV imports loaded


In [5]:
## Setup & Imports - Part 3: Transformers & Embeddings
print("Loading Transformers library... (this may take 30-60 seconds)")

from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    TrainerCallback
)
from sentence_transformers import SentenceTransformer

print("✓ Transformers library loaded")

Loading Transformers library... (this may take 30-60 seconds)
✓ Transformers library loaded
✓ Transformers library loaded


In [6]:
## Setup & Imports - Part 4: Scikit-learn & Optuna

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix
)

import optuna
from optuna.samplers import TPESampler

print("✓ Scikit-learn and Optuna loaded")

✓ Scikit-learn and Optuna loaded


In [7]:
## Configuration & Setup

# Set random seeds for reproducibility (matching LoRA approach)
SEED = 42

def set_seed(seed: int = 42):
    """Set random seed for reproducibility across all libraries."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    print(f"✓ Random seed set to {seed}")

set_seed(SEED)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Directory configuration
# Data: shared with LoRA (../data/)
# Models: local to fine_tuning/
DATA_DIR = Path('../data/processed')  # Shared data
MODELS_DIR = Path('models')           # Local models
LOGS_DIR = Path('outputs/logs')       # Local logs

# Create directories
(MODELS_DIR / 'tfidf_ffnn').mkdir(parents=True, exist_ok=True)
(MODELS_DIR / 'e5_classifier').mkdir(parents=True, exist_ok=True)
(MODELS_DIR / 'bert_finetuned').mkdir(parents=True, exist_ok=True)
LOGS_DIR.mkdir(parents=True, exist_ok=True)

# Training configuration (matching LoRA)
MAX_LENGTH = 256  # Token limit
N_TRIALS = 10     # Optuna trials
N_SPLITS = 3      # K-fold splits

print("✓ Setup complete")
print(f"  Working directory: {os.getcwd()}")
print(f"  Data directory (shared): {DATA_DIR.absolute()}")
print(f"  Models directory (local): {MODELS_DIR.absolute()}")
print(f"  Max length: {MAX_LENGTH}")
print(f"  Optuna trials: {N_TRIALS}")
print(f"  K-fold splits: {N_SPLITS}")


✓ Random seed set to 42
Using device: cuda
✓ Setup complete
  Working directory: /home3/rudrapra001/FYP-Research/export_package/fine_tuning
  Data directory (shared): /home3/rudrapra001/FYP-Research/export_package/fine_tuning/../data/processed
  Models directory (local): /home3/rudrapra001/FYP-Research/export_package/fine_tuning/models
  Max length: 256
  Optuna trials: 10
  K-fold splits: 3


---
## Part 1: Load Processed Data

Load the preprocessed datasets from Notebook 1.

---

In [8]:
## 1.1: Load Processed Data

def load_json_data(filepath):
    """Load JSON dataset"""
    with open(filepath, 'r', encoding='utf-8') as f:
        return json.load(f)

print("Loading datasets from ../data/processed/...")

# Load datasets created by first.ipynb (shared data directory)
train_data = load_json_data(DATA_DIR / 'train.json')
val_data = load_json_data(DATA_DIR / 'eval.json')

# Load summary
summary = load_json_data(DATA_DIR / 'dataset_summary.json')

print(f"\n✓ Datasets loaded:")
print(f"  Train: {len(train_data):,} samples (IMDB + Yelp)")
print(f"  Eval:  {len(val_data):,} samples (IMDB + Yelp)")

# Extract texts and labels
train_texts = [item['text'] for item in train_data]
train_labels = [item['label'] for item in train_data]

val_texts = [item['text'] for item in val_data]
val_labels = [item['label'] for item in val_data]

print("\n✓ Data extracted and ready for training")
print(f"  Train samples: {len(train_texts):,}")
print(f"  Eval samples:  {len(val_texts):,}")


Loading datasets from ../data/processed/...

✓ Datasets loaded:
  Train: 48,000 samples (IMDB + Yelp)
  Eval:  12,000 samples (IMDB + Yelp)

✓ Data extracted and ready for training
  Train samples: 48,000
  Eval samples:  12,000

✓ Datasets loaded:
  Train: 48,000 samples (IMDB + Yelp)
  Eval:  12,000 samples (IMDB + Yelp)

✓ Data extracted and ready for training
  Train samples: 48,000
  Eval samples:  12,000


In [9]:
## 1.2: Data Statistics

def print_label_distribution(labels, name):
    """Print label distribution"""
    unique, counts = np.unique(labels, return_counts=True)
    print(f"\n{name} label distribution:")
    for label, count in zip(unique, counts):
        sentiment = 'Positive' if label == 1 else 'Negative'
        print(f"  {sentiment}: {count:,} ({count/len(labels)*100:.1f}%)")

print_label_distribution(train_labels, "Training")
print_label_distribution(val_labels, "Validation")


Training label distribution:
  Negative: 24,030 (50.1%)
  Positive: 23,970 (49.9%)

Validation label distribution:
  Negative: 5,970 (49.8%)
  Positive: 6,030 (50.2%)


---
## Part 2: Model 1 - TF-IDF + Feed-Forward Neural Network

Classic approach: Convert text to TF-IDF features, then train a neural network.

---

In [10]:
## 2.1: TF-IDF Feature Extraction

print("Creating TF-IDF features...")
print("This may take 2-3 minutes...\n")

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_features=10000,      # Top 10k features
    ngram_range=(1, 2),      # Unigrams and bigrams
    min_df=5,                 # Minimum document frequency
    max_df=0.8,               # Maximum document frequency
    lowercase=True,
    strip_accents='unicode',
    stop_words='english'
)

# Fit on training data and transform
X_train_tfidf = tfidf_vectorizer.fit_transform(train_texts)
X_val_tfidf = tfidf_vectorizer.transform(val_texts)

# Convert to dense arrays for PyTorch
X_train_tfidf = X_train_tfidf.toarray()
X_val_tfidf = X_val_tfidf.toarray()

print(f"✓ TF-IDF features created:")
print(f"  Train shape: {X_train_tfidf.shape}")
print(f"  Val shape:   {X_val_tfidf.shape}")
print(f"  Vocabulary size: {len(tfidf_vectorizer.vocabulary_):,}")

# Save vectorizer
with open('models/tfidf_ffnn/tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)
print("\n✓ TF-IDF vectorizer saved")

Creating TF-IDF features...
This may take 2-3 minutes...

✓ TF-IDF features created:
  Train shape: (48000, 10000)
  Val shape:   (12000, 10000)
  Vocabulary size: 10,000
✓ TF-IDF features created:
  Train shape: (48000, 10000)
  Val shape:   (12000, 10000)
  Vocabulary size: 10,000

✓ TF-IDF vectorizer saved

✓ TF-IDF vectorizer saved


In [11]:
## 2.2: Define FFNN Model

class FFNNClassifier(nn.Module):
    """Feed-Forward Neural Network for classification"""
    
    def __init__(self, input_dim, hidden_dims, dropout_rate=0.3):
        super(FFNNClassifier, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        # Hidden layers
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            layers.append(nn.BatchNorm1d(hidden_dim))
            prev_dim = hidden_dim
        
        # Output layer
        layers.append(nn.Linear(prev_dim, 2))  # Binary classification
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

print("✓ FFNN model class defined")

✓ FFNN model class defined


In [12]:
## 2.3: FFNN Training Function

def train_ffnn(model, train_loader, val_loader, optimizer, criterion, 
               num_epochs=20, patience=3, device='cpu', save_path=None):
    """Train FFNN with early stopping
    
    Args:
        save_path: Optional path to save best model. If None, model is not saved.
    """
    
    best_val_loss = float('inf')
    best_val_acc = 0.0
    patience_counter = 0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")
        for batch_X, batch_y in pbar:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += batch_y.size(0)
            train_correct += (predicted == batch_y).sum().item()
            
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        avg_train_loss = train_loss / len(train_loader)
        train_acc = 100 * train_correct / train_total
        
        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += batch_y.size(0)
                val_correct += (predicted == batch_y).sum().item()
        
        avg_val_loss = val_loss / len(val_loader)
        val_acc = 100 * val_correct / val_total
        
        # Update history
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_acc)
        
        print(f"Epoch {epoch+1}: Train Loss={avg_train_loss:.4f}, Train Acc={train_acc:.2f}% | "
              f"Val Loss={avg_val_loss:.4f}, Val Acc={val_acc:.2f}%")
        
        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_val_loss = avg_val_loss
            patience_counter = 0
            # Save best model if save_path is provided
            if save_path is not None:
                torch.save(model.state_dict(), save_path)
                print(f"  ✓ New best model saved to {save_path} (Val Acc: {val_acc:.2f}%)")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"\nEarly stopping triggered after {epoch+1} epochs")
                break
    
    return history, best_val_acc

print("✓ Training function defined")

✓ Training function defined


In [13]:
## 2.4: Hyperparameter Optimization with Optuna + K-Fold CV

def objective_ffnn_kfold(trial):
    """Optuna objective function for FFNN with K-Fold Cross-Validation
    
    Uses 3-fold stratified CV to get more robust performance estimates.
    Returns the mean validation accuracy across all folds.
    """
    
    # Hyperparameters to optimize
    hidden_dim1 = trial.suggest_int('hidden_dim1', 128, 512, step=64)
    hidden_dim2 = trial.suggest_int('hidden_dim2', 64, 256, step=32)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    
    print(f"\n[Trial {trial.number}] Testing: hidden={hidden_dim1},{hidden_dim2}, "
          f"dropout={dropout_rate:.3f}, lr={learning_rate:.2e}, batch={batch_size}")
    
    # K-Fold Cross-Validation setup
    n_folds = 3
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=SEED)
    
    fold_scores = []
    
    # Combine train and val for k-fold (we'll use test set for final evaluation)
    X_combined = np.vstack([X_train_tfidf, X_val_tfidf])
    y_combined = np.array(train_labels + val_labels)
    
    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_combined, y_combined)):
        print(f"  Fold {fold_idx+1}/{n_folds}...", end=" ")
        
        # Split data for this fold
        X_fold_train = X_combined[train_idx]
        y_fold_train = y_combined[train_idx]
        X_fold_val = X_combined[val_idx]
        y_fold_val = y_combined[val_idx]
        
        # Create fresh model for this fold
        model = FFNNClassifier(
            input_dim=X_train_tfidf.shape[1],
            hidden_dims=[hidden_dim1, hidden_dim2],
            dropout_rate=dropout_rate
        ).to(device)
        
        # Create dataloaders
        train_dataset = torch.utils.data.TensorDataset(
            torch.FloatTensor(X_fold_train),
            torch.LongTensor(y_fold_train)
        )
        val_dataset = torch.utils.data.TensorDataset(
            torch.FloatTensor(X_fold_val),
            torch.LongTensor(y_fold_val)
        )
        
        # Use drop_last=True to avoid BatchNorm issues with single-sample batches
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, drop_last=False)
        
        # Optimizer and loss
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        criterion = nn.CrossEntropyLoss()
        
        # Train with reduced epochs for optimization (faster tuning)
        _, best_val_acc = train_ffnn(
            model, train_loader, val_loader, optimizer, criterion,
            num_epochs=10, patience=2, device=device, save_path=None
        )
        
        fold_scores.append(best_val_acc)
        print(f"Acc={best_val_acc:.2f}%")
        
        # Clean up to save memory
        del model, optimizer, train_loader, val_loader
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    # Return mean accuracy across all folds
    mean_acc = np.mean(fold_scores)
    std_acc = np.std(fold_scores)
    print(f"  → Mean CV Accuracy: {mean_acc:.2f}% (±{std_acc:.2f}%)")
    
    return mean_acc

print("✓ K-Fold CV Optuna objective function defined")

✓ K-Fold CV Optuna objective function defined


In [14]:
## 2.5: Run Hyperparameter Optimization with K-Fold CV

print("Starting K-Fold Cross-Validation hyperparameter optimization for TF-IDF + FFNN...")
print("Using 3-fold CV for robust hyperparameter selection")
print("This will take 20-40 minutes...\n")

# Create Optuna study
study_ffnn = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=SEED),
    study_name='tfidf_ffnn_kfold_optimization'
)

# Optimize with k-fold CV
study_ffnn.optimize(objective_ffnn_kfold, n_trials=15, show_progress_bar=True)

# Best parameters
print("\n" + "="*70)
print("K-FOLD CV OPTIMIZATION COMPLETE - TF-IDF + FFNN")
print("="*70)
print(f"Best mean CV accuracy: {study_ffnn.best_value:.2f}%")
print(f"\nBest hyperparameters:")
for key, value in study_ffnn.best_params.items():
    print(f"  {key}: {value}")

# Save study results
os.makedirs('models/tfidf_ffnn', exist_ok=True)
with open('models/tfidf_ffnn/optuna_study_kfold.pkl', 'wb') as f:
    pickle.dump(study_ffnn, f)
print("\n✓ K-Fold CV optimization results saved")

[I 2025-11-10 00:56:00,875] A new study created in memory with name: tfidf_ffnn_kfold_optimization


Starting K-Fold Cross-Validation hyperparameter optimization for TF-IDF + FFNN...
Using 3-fold CV for robust hyperparameter selection
This will take 20-40 minutes...



  0%|          | 0/15 [00:00<?, ?it/s]


[Trial 0] Testing: hidden=256,256, dropout=0.420, lr=1.58e-03, batch=32
  Fold 1/3...   Fold 1/3... 

Epoch 1/10 [Train]: 100%|██████████| 1250/1250 [00:16<00:00, 74.97it/s, loss=0.3019]



...

Epoch 2: Train Loss=0.1730, Train Acc=93.30% | Val Loss=0.2912, Val Acc=88.72%


Epoch 3/10 [Train]: 100%|██████████| 625/625 [00:09<00:00, 67.42it/s, loss=0.0581]


Epoch 3: Train Loss=0.0765, Train Acc=97.25% | Val Loss=0.4010, Val Acc=88.31%

Early stopping triggered after 3 epochs
Acc=89.17%
  → Mean CV Accuracy: 89.13% (±0.16%)


Best trial: 11. Best value: 89.44: 100%|██████████| 15/15 [32:14<00:00, 128.95s/it]

[I 2025-11-10 01:28:15,190] Trial 14 finished with value: 89.13333333333334 and parameters: {'hidden_dim1': 384, 'hidden_dim2': 64, 'dropout_rate': 0.33865304013034375, 'learning_rate': 0.000529056994167218, 'batch_size': 64}. Best is trial 11 with value: 89.44.

K-FOLD CV OPTIMIZATION COMPLETE - TF-IDF + FFNN
Best mean CV accuracy: 89.44%

Best hyperparameters:
  hidden_dim1: 320
  hidden_dim2: 64
  dropout_rate: 0.3535602771439335
  learning_rate: 0.00011835846181383978
  batch_size: 64

✓ K-Fold CV optimization results saved


In [15]:
## 2.6: Train Final FFNN Model with Best Hyperparameters

print("Training final TF-IDF + FFNN model with best hyperparameters...\n")

best_params = study_ffnn.best_params

# Create final model
final_ffnn_model = FFNNClassifier(
    input_dim=X_train_tfidf.shape[1],
    hidden_dims=[best_params['hidden_dim1'], best_params['hidden_dim2']],
    dropout_rate=best_params['dropout_rate']
).to(device)

# Create dataloaders
train_dataset = torch.utils.data.TensorDataset(
    torch.FloatTensor(X_train_tfidf),
    torch.LongTensor(train_labels)
)
val_dataset = torch.utils.data.TensorDataset(
    torch.FloatTensor(X_val_tfidf),
    torch.LongTensor(val_labels)
)

train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_params['batch_size'])

# Optimizer and loss
optimizer = optim.Adam(final_ffnn_model.parameters(), lr=best_params['learning_rate'])
criterion = nn.CrossEntropyLoss()

# Train full model
history_ffnn, best_val_acc = train_ffnn(
    final_ffnn_model, train_loader, val_loader, optimizer, criterion,
    num_epochs=30, patience=5, device=device, save_path='models/tfidf_ffnn/best_model.pt'
)

# Save training history
with open('models/tfidf_ffnn/training_history.pkl', 'wb') as f:
    pickle.dump(history_ffnn, f)

# Save hyperparameters
with open('models/tfidf_ffnn/hyperparameters.json', 'w') as f:
    json.dump(best_params, f, indent=2)

print("\n" + "="*70)
print("TF-IDF + FFNN TRAINING COMPLETE")
print("="*70)
print(f"Best validation accuracy: {best_val_acc:.2f}%")
print("\n✓ Model saved to: models/tfidf_ffnn/")

Training final TF-IDF + FFNN model with best hyperparameters...



Epoch 1/30 [Train]: 100%|██████████| 750/750 [00:13<00:00, 55.79it/s, loss=0.1832]



Epoch 1: Train Loss=0.3527, Train Acc=84.03% | Val Loss=0.2603, Val Acc=89.26%
  ✓ New best model saved to models/tfidf_ffnn/best_model.pt (Val Acc: 89.26%)


Epoch 2/30 [Train]: 100%|██████████| 750/750 [00:09<00:00, 75.25it/s, loss=0.1938] 



Epoch 2: Train Loss=0.1891, Train Acc=92.87% | Val Loss=0.2723, Val Acc=88.61%


...

Epoch 4: Train Loss=0.0508, Train Acc=98.40% | Val Loss=0.3830, Val Acc=87.95%


Epoch 5/30 [Train]: 100%|██████████| 750/750 [00:14<00:00, 50.29it/s, loss=0.0356]


Epoch 5: Train Loss=0.0288, Train Acc=99.17% | Val Loss=0.4313, Val Acc=87.92%


Epoch 6/30 [Train]: 100%|██████████| 750/750 [00:11<00:00, 63.86it/s, loss=0.0050]


Epoch 6: Train Loss=0.0211, Train Acc=99.33% | Val Loss=0.4624, Val Acc=87.96%

Early stopping triggered after 6 epochs

TF-IDF + FFNN TRAINING COMPLETE
Best validation accuracy: 89.26%

✓ Model saved to: models/tfidf_ffnn/


In [16]:
## 3.1: Generate E5 Embeddings

print("Loading E5 embedding model...")
print("This will download ~400MB on first run...\n")

# Load E5 model (or use MiniLM as alternative)
# E5 is stronger but larger, MiniLM is faster
embedding_model_name = 'intfloat/e5-small-v2'  # or 'sentence-transformers/all-MiniLM-L6-v2'
embedding_model = SentenceTransformer(embedding_model_name, device=device)

print(f"✓ Loaded: {embedding_model_name}")
print(f"  Embedding dimension: {embedding_model.get_sentence_embedding_dimension()}")

# Generate embeddings
print("\nGenerating embeddings...")
print("This may take 5-10 minutes...\n")

# For E5, prefix with "query: " for better performance
if 'e5' in embedding_model_name.lower():
    train_texts_prefixed = [f"query: {text}" for text in train_texts]
    val_texts_prefixed = [f"query: {text}" for text in val_texts]
else:
    train_texts_prefixed = train_texts
    val_texts_prefixed = val_texts

# Generate embeddings in batches
X_train_embeddings = embedding_model.encode(
    train_texts_prefixed,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True
)

X_val_embeddings = embedding_model.encode(
    val_texts_prefixed,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True
)

print(f"\n✓ Embeddings generated:")
print(f"  Train shape: {X_train_embeddings.shape}")
print(f"  Val shape:   {X_val_embeddings.shape}")

# Save embeddings (for faster re-runs)
np.save('models/e5_classifier/train_embeddings.npy', X_train_embeddings)
np.save('models/e5_classifier/val_embeddings.npy', X_val_embeddings)
print("\n✓ Embeddings saved")

Loading E5 embedding model...
This will download ~400MB on first run...

✓ Loaded: intfloat/e5-small-v2
  Embedding dimension: 384

Generating embeddings...
This may take 5-10 minutes...

✓ Loaded: intfloat/e5-small-v2
  Embedding dimension: 384

Generating embeddings...
This may take 5-10 minutes...



Batches: 100%|██████████| 750/750 [02:06<00:00,  5.94it/s]

Batches: 100%|██████████| 188/188 [00:34<00:00,  5.45it/s]




✓ Embeddings generated:
  Train shape: (48000, 384)
  Val shape:   (12000, 384)

✓ Embeddings saved


In [17]:
## 3.2: Define Classifier Head for Embeddings

class EmbeddingClassifier(nn.Module):
    """Simple classifier head for pre-computed embeddings"""
    
    def __init__(self, embedding_dim, hidden_dim, dropout_rate=0.3):
        super(EmbeddingClassifier, self).__init__()
        
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 2)  # Binary classification
        )
    
    def forward(self, x):
        return self.classifier(x)

print("✓ Embedding classifier class defined")

✓ Embedding classifier class defined


In [18]:
## 3.3: Hyperparameter Optimization for Embedding Classifier with K-Fold CV

def objective_embedding_kfold(trial):
    """Optuna objective for embedding classifier with K-Fold CV"""
    
    # Hyperparameters
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256, step=32)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    
    print(f"\n[Trial {trial.number}] Testing: hidden={hidden_dim}, "
          f"dropout={dropout_rate:.3f}, lr={learning_rate:.2e}, batch={batch_size}")
    
    # K-Fold Cross-Validation setup
    n_folds = 3
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=SEED)
    
    fold_scores = []
    
    # Combine train and val embeddings for k-fold
    X_combined = np.vstack([X_train_embeddings, X_val_embeddings])
    y_combined = np.array(train_labels + val_labels)
    
    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_combined, y_combined)):
        print(f"  Fold {fold_idx+1}/{n_folds}...", end=" ")
        
        # Split data for this fold
        X_fold_train = X_combined[train_idx]
        y_fold_train = y_combined[train_idx]
        X_fold_val = X_combined[val_idx]
        y_fold_val = y_combined[val_idx]
        
        # Create fresh model for this fold
        model = EmbeddingClassifier(
            embedding_dim=X_train_embeddings.shape[1],
            hidden_dim=hidden_dim,
            dropout_rate=dropout_rate
        ).to(device)
        
        # Create dataloaders
        train_dataset = torch.utils.data.TensorDataset(
            torch.FloatTensor(X_fold_train),
            torch.LongTensor(y_fold_train)
        )
        val_dataset = torch.utils.data.TensorDataset(
            torch.FloatTensor(X_fold_val),
            torch.LongTensor(y_fold_val)
        )
        
        # Use drop_last=True to avoid BatchNorm issues with single-sample batches
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, drop_last=False)
        
        # Optimizer and loss
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        criterion = nn.CrossEntropyLoss()
        
        # Train (reuse training function)
        _, best_val_acc = train_ffnn(
            model, train_loader, val_loader, optimizer, criterion,
            num_epochs=10, patience=2, device=device, save_path=None
        )
        
        fold_scores.append(best_val_acc)
        print(f"Acc={best_val_acc:.2f}%")
        
        # Clean up to save memory
        del model, optimizer, train_loader, val_loader
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    # Return mean accuracy across all folds
    mean_acc = np.mean(fold_scores)
    std_acc = np.std(fold_scores)
    print(f"  → Mean CV Accuracy: {mean_acc:.2f}% (±{std_acc:.2f}%)")
    
    return mean_acc

print("Starting K-Fold CV hyperparameter optimization for E5 classifier...")
print("Using 3-fold CV for robust hyperparameter selection")
print("This will take 15-25 minutes...\n")

# Create study
study_embedding = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=SEED),
    study_name='embedding_classifier_kfold_optimization'
)

# Optimize with k-fold CV
study_embedding.optimize(objective_embedding_kfold, n_trials=12, show_progress_bar=True)

print("\n" + "="*70)
print("K-FOLD CV OPTIMIZATION COMPLETE - E5 Embedding Classifier")
print("="*70)
print(f"Best mean CV accuracy: {study_embedding.best_value:.2f}%")
print(f"\nBest hyperparameters:")
for key, value in study_embedding.best_params.items():
    print(f"  {key}: {value}")

# Save study
os.makedirs('models/e5_classifier', exist_ok=True)
with open('models/e5_classifier/optuna_study_kfold.pkl', 'wb') as f:
    pickle.dump(study_embedding, f)
print("\n✓ K-Fold CV optimization results saved")

[I 2025-11-10 01:32:34,384] A new study created in memory with name: embedding_classifier_kfold_optimization


Starting K-Fold CV hyperparameter optimization for E5 classifier...
Using 3-fold CV for robust hyperparameter selection
This will take 15-25 minutes...



  0%|          | 0/12 [00:00<?, ?it/s]


[Trial 0] Testing: hidden=128, dropout=0.485, lr=2.91e-03, batch=32
  Fold 1/3... 

Epoch 1/10 [Train]: 100%|██████████| 1250/1250 [00:16<00:00, 76.76it/s, loss=0.1688]


...

Epoch 4: Train Loss=0.1578, Train Acc=94.03% | Val Loss=0.1581, Val Acc=94.12%


Epoch 5/10 [Train]: 100%|██████████| 625/625 [00:07<00:00, 78.31it/s, loss=0.1753]


Epoch 5: Train Loss=0.1581, Train Acc=93.92% | Val Loss=0.1559, Val Acc=94.11%


Best trial: 4. Best value: 94.255: 100%|██████████| 12/12 [34:03<00:00, 170.27s/it]

Epoch 6: Train Loss=0.1537, Train Acc=94.16% | Val Loss=0.1587, Val Acc=94.08%

Early stopping triggered after 6 epochs
Acc=94.12%
  → Mean CV Accuracy: 94.23% (±0.08%)
[I 2025-11-10 02:06:37,570] Trial 11 finished with value: 94.23333333333333 and parameters: {'hidden_dim': 192, 'dropout_rate': 0.3548125484709944, 'learning_rate': 0.00010744583847151158, 'batch_size': 64}. Best is trial 4 with value: 94.255.

K-FOLD CV OPTIMIZATION COMPLETE - E5 Embedding Classifier
Best mean CV accuracy: 94.25%

Best hyperparameters:
  hidden_dim: 160
  dropout_rate: 0.43555278841790407
  learning_rate: 0.00025081156860452336
  batch_size: 64

✓ K-Fold CV optimization results saved


In [19]:
## 3.4: Train Final E5 Classifier

print("Training final E5 classifier with best hyperparameters...\n")

best_params = study_embedding.best_params

# Create final model
final_embedding_model = EmbeddingClassifier(
    embedding_dim=X_train_embeddings.shape[1],
    hidden_dim=best_params['hidden_dim'],
    dropout_rate=best_params['dropout_rate']
).to(device)

# Create dataloaders
train_dataset = torch.utils.data.TensorDataset(
    torch.FloatTensor(X_train_embeddings),
    torch.LongTensor(train_labels)
)
val_dataset = torch.utils.data.TensorDataset(
    torch.FloatTensor(X_val_embeddings),
    torch.LongTensor(val_labels)
)

train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_params['batch_size'])

# Optimizer and loss
optimizer = optim.Adam(final_embedding_model.parameters(), lr=best_params['learning_rate'])
criterion = nn.CrossEntropyLoss()

# Train
history_embedding, best_val_acc = train_ffnn(
    final_embedding_model, train_loader, val_loader, optimizer, criterion,
    num_epochs=30, patience=5, device=device, save_path='models/e5_classifier/best_model.pt'
)

# Save everything
with open('models/e5_classifier/training_history.pkl', 'wb') as f:
    pickle.dump(history_embedding, f)

with open('models/e5_classifier/hyperparameters.json', 'w') as f:
    json.dump(best_params, f, indent=2)

# Save model name for later use
with open('models/e5_classifier/embedding_model_name.txt', 'w') as f:
    f.write(embedding_model_name)

print("\n" + "="*70)
print("E5 EMBEDDING CLASSIFIER TRAINING COMPLETE")
print("="*70)
print(f"Best validation accuracy: {best_val_acc:.2f}%")
print("\n✓ Model saved to: models/e5_classifier/")

Training final E5 classifier with best hyperparameters...



Epoch 1/30 [Train]: 100%|██████████| 750/750 [00:07<00:00, 99.50it/s, loss=0.1811] 


Epoch 1: Train Loss=0.1905, Train Acc=92.70% | Val Loss=0.1578, Val Acc=94.08%
  ✓ New best model saved to models/e5_classifier/best_model.pt (Val Acc: 94.08%)


Epoch 2/30 [Train]: 100%|██████████| 750/750 [00:07<00:00, 98.17it/s, loss=0.1213] 


Epoch 2: Train Loss=0.1683, Train Acc=93.70% | Val Loss=0.1601, Val Acc=93.69%


...

Epoch 4: Train Loss=0.1625, Train Acc=93.79% | Val Loss=0.1529, Val Acc=94.05%


Epoch 5/30 [Train]: 100%|██████████| 750/750 [00:11<00:00, 63.30it/s, loss=0.1679]


Epoch 5: Train Loss=0.1592, Train Acc=93.96% | Val Loss=0.1510, Val Acc=94.08%


Epoch 6/30 [Train]: 100%|██████████| 750/750 [00:06<00:00, 110.45it/s, loss=0.1956]


Epoch 6: Train Loss=0.1587, Train Acc=93.92% | Val Loss=0.1535, Val Acc=94.04%

Early stopping triggered after 6 epochs

E5 EMBEDDING CLASSIFIER TRAINING COMPLETE
Best validation accuracy: 94.08%

✓ Model saved to: models/e5_classifier/


---
## Part 4: Model 3 - BERT Fine-Tuning with K-Fold CV Optimization

Fine-tune BERT (bert-base-uncased) for sentiment classification.
Uses K-Fold CV to find optimal hyperparameters for better performance.

**Key Improvements:**
- 2-fold stratified CV for hyperparameter search (faster than 3-fold for BERT)
- Optimizes: learning rate, weight decay, warmup ratio, epochs, batch size
- More robust than fixed hyperparameters

---

In [20]:
## 4.1: Prepare BERT Dataset

class SentimentDataset(Dataset):
    """Dataset class for BERT tokenization"""
    
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

print("✓ BERT dataset class defined")

✓ BERT dataset class defined


In [1]:
## 4.2: Load BERT Tokenizer and Model

print("Loading BERT model and tokenizer...")
print("This will download ~440MB on first run...\n")

model_name = 'bert-base-uncased'

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model for sequence classification
bert_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False
)

print(f"✓ Loaded: {model_name}")
print(f"  Total parameters: {sum(p.numel() for p in bert_model.parameters()):,}")
print(f"  Trainable parameters: {sum(p.numel() for p in bert_model.parameters() if p.requires_grad):,}")

Loading BERT model and tokenizer...
This will download ~440MB on first run...



NameError: name 'AutoTokenizer' is not defined

In [ ]:
## 4.3: Create BERT Datasets

print("Creating tokenized datasets...")
print("This may take 2-3 minutes...\n")

# Create datasets
train_dataset_bert = SentimentDataset(
    texts=train_texts,
    labels=train_labels,
    tokenizer=tokenizer,
    max_length=256
)

val_dataset_bert = SentimentDataset(
    texts=val_texts,
    labels=val_labels,
    tokenizer=tokenizer,
    max_length=256
)

print(f"✓ Datasets created:")
print(f"  Train: {len(train_dataset_bert):,} samples")
print(f"  Val:   {len(val_dataset_bert):,} samples")

# Test tokenization
sample = train_dataset_bert[0]
print(f"\nSample tokenization:")
print(f"  Input IDs shape: {sample['input_ids'].shape}")
print(f"  Attention mask shape: {sample['attention_mask'].shape}")
print(f"  Label: {sample['labels']}")

Creating tokenized datasets...
This may take 2-3 minutes...

✓ Datasets created:
  Train: 48,000 samples
  Val:   12,000 samples

Sample tokenization:
  Input IDs shape: torch.Size([256])
  Attention mask shape: torch.Size([256])
  Label: 0


In [ ]:
## 4.4: Define Metrics for BERT Training

def compute_metrics(eval_pred):
    """Compute metrics for BERT evaluation"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='binary'
    )
    acc = accuracy_score(labels, predictions)
    
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

print("✓ Metrics function defined")

✓ Metrics function defined


In [ ]:
%pip install 'accelerate>={ACCELERATE_MIN_VERSION}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 1: syntax error: unexpected end of file
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
## 4.5: BERT Hyperparameter Optimization with K-Fold CV

def objective_bert_kfold(trial):
    """Optuna objective for BERT with K-Fold CV
    
    Uses stratified 2-fold CV (not 3-fold due to BERT's computational cost).
    Optimizes key hyperparameters that affect BERT performance.
    """
    
    # Hyperparameters to optimize
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 5e-5, log=True)
    weight_decay = trial.suggest_float('weight_decay', 0.0, 0.1)
    warmup_ratio = trial.suggest_float('warmup_ratio', 0.0, 0.2)
    num_epochs = trial.suggest_int('num_epochs', 2, 4)
    batch_size = trial.suggest_categorical('batch_size', [8, 16])
    
    print(f"\n[Trial {trial.number}] Testing: lr={learning_rate:.2e}, wd={weight_decay:.3f}, "
          f"warmup={warmup_ratio:.2f}, epochs={num_epochs}, batch={batch_size}")
    
    # Use 2-fold CV for BERT (computational efficiency)
    n_folds = 2
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=SEED)
    
    fold_scores = []
    
    # Combine train and val for k-fold
    combined_texts = train_texts + val_texts
    combined_labels = train_labels + val_labels
    
    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(combined_texts, combined_labels)):
        print(f"  Fold {fold_idx+1}/{n_folds}...", end=" ")
        
        # Create datasets for this fold
        fold_train_texts = [combined_texts[i] for i in train_idx]
        fold_train_labels = [combined_labels[i] for i in train_idx]
        fold_val_texts = [combined_texts[i] for i in val_idx]
        fold_val_labels = [combined_labels[i] for i in val_idx]
        
        fold_train_dataset = SentimentDataset(
            texts=fold_train_texts,
            labels=fold_train_labels,
            tokenizer=tokenizer,
            max_length=256
        )
        fold_val_dataset = SentimentDataset(
            texts=fold_val_texts,
            labels=fold_val_labels,
            tokenizer=tokenizer,
            max_length=256
        )
        
        # Load fresh BERT model for this fold
        fold_model = AutoModelForSequenceClassification.from_pretrained(
            'bert-base-uncased',
            num_labels=2,
            output_attentions=False,
            output_hidden_states=False
        )
        
        # Training arguments for this fold
        fold_training_args = TrainingArguments(
            output_dir=f'models/bert_finetuned/trial_{trial.number}_fold_{fold_idx}',
            num_train_epochs=num_epochs,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=32,
            warmup_ratio=warmup_ratio,
            weight_decay=weight_decay,
            learning_rate=learning_rate,
            logging_steps=200,
            evaluation_strategy='epoch',
            save_strategy='epoch',
            save_total_limit=1,
            load_best_model_at_end=True,
            metric_for_best_model='f1',
            greater_is_better=True,
            fp16=torch.cuda.is_available(),
            seed=SEED,
            report_to='none',  # Disable wandb/tensorboard during tuning
            logging_dir=None
        )
        
        # Create trainer
        fold_trainer = Trainer(
            model=fold_model,
            args=fold_training_args,
            train_dataset=fold_train_dataset,
            eval_dataset=fold_val_dataset,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
        )
        
        # Train
        fold_trainer.train()
        
        # Evaluate
        eval_results = fold_trainer.evaluate()
        fold_f1 = eval_results['eval_f1']
        fold_scores.append(fold_f1)
        
        print(f"F1={fold_f1:.4f}")
        
        # Clean up
        del fold_model, fold_trainer
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
        
        # Remove temporary checkpoint directory
        import shutil
        if os.path.exists(f'models/bert_finetuned/trial_{trial.number}_fold_{fold_idx}'):
            shutil.rmtree(f'models/bert_finetuned/trial_{trial.number}_fold_{fold_idx}')
    
    # Return mean F1 score across folds
    mean_f1 = np.mean(fold_scores)
    std_f1 = np.std(fold_scores)
    print(f"  → Mean CV F1: {mean_f1:.4f} (±{std_f1:.4f})")
    
    return mean_f1

print("Starting K-Fold CV hyperparameter optimization for BERT...")
print("Using 2-fold CV for computational efficiency")
print("This will take 1-3 hours depending on your hardware...\n")
print("NOTE: You can reduce n_trials to 3-5 for faster testing\n")

# Create study
study_bert = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=SEED),
    study_name='bert_kfold_optimization'
)

# Optimize with k-fold CV (fewer trials due to computational cost)
study_bert.optimize(objective_bert_kfold, n_trials=8, show_progress_bar=True)

print("\n" + "="*70)
print("K-FOLD CV OPTIMIZATION COMPLETE - BERT")
print("="*70)
print(f"Best mean CV F1: {study_bert.best_value:.4f}")
print(f"\nBest hyperparameters:")
for key, value in study_bert.best_params.items():
    print(f"  {key}: {value}")

# Save study results
os.makedirs('models/bert_finetuned', exist_ok=True)
with open('models/bert_finetuned/optuna_study_kfold.pkl', 'wb') as f:
    pickle.dump(study_bert, f)

# Save best params
with open('models/bert_finetuned/best_hyperparameters.json', 'w') as f:
    json.dump(study_bert.best_params, f, indent=2)

print("\n✓ K-Fold CV optimization results saved")

[I 2025-11-10 02:07:44,486] A new study created in memory with name: bert_kfold_optimization


Starting K-Fold CV hyperparameter optimization for BERT...
Using 2-fold CV for computational efficiency
This will take 1-3 hours depending on your hardware...

NOTE: You can reduce n_trials to 3-5 for faster testing



  0%|          | 0/8 [00:00<?, ?it/s]


[Trial 0] Testing: lr=1.83e-05, wd=0.095, warmup=0.15, epochs=3, batch=8
  Fold 1/2...   Fold 1/2... 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  0%|          | 0/8 [00:10<?, ?it/s]



[W 2025-11-10 02:07:55,203] Trial 0 failed with parameters: {'learning_rate': 1.827226177606625e-05, 'weight_decay': 0.09507143064099162, 'warmup_ratio': 0.146398788362281, 'num_epochs': 3, 'batch_size': 8} because of the following error: OutOfMemoryError('Caught OutOfMemoryError in replica 1 on device 1.\nOriginal Traceback (most recent call last):\n  File "/home3/rudrapra001/FYP-Research/.venv/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 84, in _worker\n    output = module(*input, **kwargs)\n  File "/home3/rudrapra001/FYP-Research/.venv/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1553, in _wrapped_call_impl\n    return self._call_impl(*args, **kwargs)\n  File "/home3/rudrapra001/FYP-Research/.venv/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1562, in _call_impl\n    return forward_call(*args, **kwargs)\n  File "/home3/rudrapra001/FYP-Research/.venv/lib/python3.8/site-packages/transformers/models/bert/modeling_bert.py", line

OutOfMemoryError: Caught OutOfMemoryError in replica 1 on device 1.
Original Traceback (most recent call last):
  File "/home3/rudrapra001/FYP-Research/.venv/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 84, in _worker
    output = module(*input, **kwargs)
  File "/home3/rudrapra001/FYP-Research/.venv/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home3/rudrapra001/FYP-Research/.venv/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1562, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home3/rudrapra001/FYP-Research/.venv/lib/python3.8/site-packages/transformers/models/bert/modeling_bert.py", line 1668, in forward
    outputs = self.bert(
  File "/home3/rudrapra001/FYP-Research/.venv/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home3/rudrapra0

In [ ]:
## 4.6: Train Final BERT Model with Best Hyperparameters

print("\nTraining final BERT model with best hyperparameters from K-Fold CV...")
print("This will take 30-60 minutes (or 2-4 hours on CPU)...\n")

best_params = study_bert.best_params

# Calculate warmup steps from warmup ratio
total_steps = (len(train_dataset_bert) // best_params['batch_size']) * best_params['num_epochs']
warmup_steps = int(total_steps * best_params['warmup_ratio'])

print(f"Using optimized hyperparameters:")
print(f"  Learning rate: {best_params['learning_rate']:.2e}")
print(f"  Weight decay: {best_params['weight_decay']:.3f}")
print(f"  Warmup ratio: {best_params['warmup_ratio']:.2f} ({warmup_steps} steps)")
print(f"  Epochs: {best_params['num_epochs']}")
print(f"  Batch size: {best_params['batch_size']}\n")

# Load fresh BERT model for final training
bert_model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False
)

# Training arguments with optimized hyperparameters
training_args = TrainingArguments(
    output_dir='models/bert_finetuned',
    num_train_epochs=best_params['num_epochs'],
    per_device_train_batch_size=best_params['batch_size'],
    per_device_eval_batch_size=32,
    warmup_steps=warmup_steps,
    weight_decay=best_params['weight_decay'],
    learning_rate=best_params['learning_rate'],
    logging_dir='outputs/training_logs/bert',
    logging_steps=100,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    seed=SEED
)

# Create Trainer
trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_dataset_bert,
    eval_dataset=val_dataset_bert,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Train
train_result = trainer.train()

# Save model
trainer.save_model('models/bert_finetuned/final_model')
tokenizer.save_pretrained('models/bert_finetuned/final_model')

# Save training metrics
with open('models/bert_finetuned/training_metrics.json', 'w') as f:
    json.dump(train_result.metrics, f, indent=2)

print("\n" + "="*70)
print("BERT FINE-TUNING COMPLETE")
print("="*70)
print(f"Training loss: {train_result.metrics['train_loss']:.4f}")
print("\n✓ Model saved to: models/bert_finetuned/final_model/")

In [ ]:
## 4.7: Evaluate BERT on Validation Set

print("Evaluating BERT on validation set...\n")

eval_results = trainer.evaluate()

print("Validation Results:")
print(f"  Accuracy:  {eval_results['eval_accuracy']:.4f}")
print(f"  F1 Score:  {eval_results['eval_f1']:.4f}")
print(f"  Precision: {eval_results['eval_precision']:.4f}")
print(f"  Recall:    {eval_results['eval_recall']:.4f}")

# Save evaluation results
with open('models/bert_finetuned/validation_metrics.json', 'w') as f:
    json.dump(eval_results, f, indent=2)

print("\n✓ Evaluation complete")

---
## ✅ Notebook 2 Complete!

### What we accomplished:
1. ✓ Trained TF-IDF + FFNN classifier with Optuna optimization
2. ✓ Trained E5 embedding classifier with Optuna optimization
3. ✓ Fine-tuned BERT for sentiment analysis
4. ✓ Saved all models and hyperparameters
5. ✓ Achieved strong validation performance on multi-domain data

### Models Saved:
- `models/tfidf_ffnn/` - TF-IDF + FFNN classifier
- `models/e5_classifier/` - E5 embedding classifier
- `models/bert_finetuned/` - Fine-tuned BERT model

### Next Steps:
→ **Notebook 3**: Evaluation & Domain Adaptation Analysis
- Load all trained models
- In-domain evaluation (IMDB + Yelp test sets)
- **Cross-domain evaluation** (Amazon test set)
- Performance comparison and visualization

---